In [6]:
import joblib # to load models


import sys
sys.path.append("../scripts") # to load function that separated data
from experiments import load_split


from credit_pipeline.training import ks_threshold


In [8]:
fold = 0
dataset = "german"
model_name = "LGBMClassifier"

model = joblib.load(f"../results/credit_models_unaware/{dataset}/{fold}/{model_name}.pkl")
model

Pipeline(steps=[('fill',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Duration', 'CreditAmount',
                                                   'InstallmentRate',
                                                   'ResidenceSince', 'Age',
                                                   'ExistingCredits',
                                                   'Dependents']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['Telephone',
                                                   'ForeignWorker']),
                                                 ('ebe',
                                                  Pipeline(steps=[('imputer...
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Telephone',
                                                   'ForeignWorker'])],
                                   verbose_feature_names_out=False)),
                ('classifier',
                 LGBMClassifier(colsample_bytree=0.4293321583431199,
                                learning_rate=0.12071307986281193, max_depth=12,
                                min_child_samples=16, n_estimators=49,
                                num_leaves=189, reg_alpha=0.021549101314163266,
                                reg_lambda=0.0014105648831451857,
                                verbose=-1))])

The models does not have the threshold saved in them. It's necessary to calculate again. Let's load the data.

In [9]:
X_train, A_train, Y_train, X_val, A_val, Y_val, X_test, A_test, Y_test = load_split(dataset, fold, unaware = True)
X_train.head()

,CheckingAccount,Duration,CreditHistory,Purpose,CreditAmount,SavingsAccount,EmploymentSince,InstallmentRate,OtherDebtors,ResidenceSince,Property,Age,OtherInstallmentPlans,Housing,ExistingCredits,Job,Dependents,Telephone,ForeignWorker
948,No,24,Existing paid,Radio/television,3621,100 - 500,> 7,2,No,4,Car or other,31,No,Own,2,Skilled,1,0,1
847,< 0,9,Existing paid,Radio/television,1364,< 100,4 - 7,3,No,4,Real estate,59,No,Own,1,Skilled,1,0,1
106,0 - 200,12,Existing paid,Car (new),6078,< 100,4 - 7,2,No,2,Car or other,32,No,Own,1,Skilled,1,0,1
239,< 0,24,Existing paid,Car (new),915,Unknown/None,> 7,4,No,2,Car or other,29,Bank,Own,1,Skilled,1,0,1
518,No,6,Critical account,Radio/television,700,Unknown/None,> 7,4,No,4,Unknown/None,36,No,For free,2,Skilled,1,0,1


In [10]:
Y_val_prob = model.predict_proba(X_val)[:,1]
thresh = ks_threshold(Y_val, Y_val_prob)
thresh

0.452052398374445